# Construcción de una Base de Datos Vectorial desde cero

Una empresa llamada DataFind quiere crear su propia base de datos vectorial interna, sin depender de servicios externos.
Necesitan almacenar, indexar y buscar información basada en significado, no solo por coincidencia de texto.

Tu misión es simular una BDV completa capaz de:

Almacenar embeddings.

Buscar por similitud.

Implementar filtros por metadatos.

Medir la eficiencia y calidad de los resultados.

## Objetivos de la práctica

1. Generar embeddings simulados

- Cargar csv documentos_datafind.csv.

- Convertirlos en vectores de alta dimensión (usando numpy para simular embeddings).

2. Implementar componentes internos de la BDV:

- Storage Engine: guardar los vectores y metadatos.

- Index Builder: crear índices con estrategias de agrupamiento (simular IVF, HNSW o PQ).

- Query Engine: calcular similitudes (cosine y euclidean).

- Metadata Store: almacenar texto original, etiquetas o categorías.

3. Implementar consultas vectoriales

- Permitir que el usuario escriba una consulta y el sistema devuelva los documentos más similares.

- Simular embeddings de la consulta con el mismo método.

4. Agregar filtrado por metadatos

- Permitir buscar solo en documentos de una categoría (ej. “Tecnología”, “Salud”, “Educación”).

5. Evaluar el rendimiento y calidad de la búsqueda:

- Calcular tiempo de consulta.

- Evaluar qué métrica (cosine o euclidean) devuelve resultados más relevantes.

6. Simular optimización del índice:

- Comparar tiempos de búsqueda con distintas estrategias (sin índice, con IVF, con HNSW).

- Analizar cuál ofrece el mejor equilibrio entre velocidad y precisión.

7. Generar un informe final automático:

- Guardar resultados de las consultas, métricas y tiempos en un archivo reporte_resultados.csv.

## SOLUCIÓN 1

Cargamos

In [1]:
import pandas as pd
import numpy as np


In [2]:
csv_path = "./Data/documentos_datafind.csv"
df = pd.read_csv(csv_path)

In [3]:
print(df.head())

   id   categoria                   titulo  \
0   1  Tecnología            Avances en IA   
1   2       Salud      Nuevos tratamientos   
2   3   Educación   Aprendizaje automático   
3   4    Economía        Mercados globales   
4   5     Ciencia  Descubrimiento espacial   

                                           contenido  
0  La inteligencia artificial está transformando ...  
1  Se desarrollan terapias genéticas para enferme...  
2  El machine learning se utiliza en plataformas ...  
3  Los mercados bursátiles muestran volatilidad a...  
4  Un nuevo telescopio detecta exoplanetas simila...  


Convertir en vectores de alta dimensión == crear embeddings

Rápido sin dependencias pero no tiene significado semántico

In [4]:
embedding_dim = 512  # por ejemplo, 512 dimensiones

In [5]:
# Generamos un embedding aleatorio para cada documento
# np.random.rand genera valores entre 0 y 1
embeddings = np.random.rand(len(df), embedding_dim)

# Añadimos los embeddings al DataFrame si quieres guardarlos
df['embedding'] = embeddings.tolist()


TF-IDF: es una métrica para medir la importancia de una palabra en un documento dentro de una colección de documentos.

Se calcula multiplicando dos factores: la frecuencia con la que aparece una palabra en un documento (TF) y la rareza de esa palabra en todos los documentos del corpus (IDF). Un valor alto de TF-IDF indica que la palabra es relevante para ese documento específico. 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk

In [ ]:
csv_path = "./Data/documentos_datafind.csv"
df = pd.read_csv(csv_path)

In [ ]:
nltk.download('stopwords')

In [ ]:
stop_words_es = stopwords.words('spanish')

In [ ]:
df["texto_completo"] = df["titulo"].astype(str) + ". " + df["contenido"].astype(str)

# Creamos vectorizador TF-IDF con stopwords en español
vectorizer = TfidfVectorizer(
    max_features=512,
    stop_words=stop_words_es
)

In [ ]:
embeddings = vectorizer.fit_transform(df["texto_completo"])
df["embedding"] = embeddings.toarray().tolist()

print(df[["id", "categoria", "titulo"]].head())

## SOLUCIÓN 2

In [6]:
class StorageEngine:
    def __init__(self):
        self.vectors = None
        self.metadata = None

    def store(self, vectors, metadata):
        self.vectors = vectors
        self.metadata = metadata
        print("Vectores y metadatos guardados en StorageEngine.")

## 1. IVF (Inverted File)

La idea de IVF es **agrupar vectores en clusters** y buscar solo dentro del cluster más cercano:

**Fórmula del cluster más cercano:**

$$
cluster\_id = argmin_{c \in C} || v - \mu_c ||
$$

Donde:  
- v = vector de embedding del documento  
- μ_c = centroide del cluster c  
- C = conjunto de todos los clusters  




## 2. HNSW (Hierarchical Navigable Small World)

HNSW construye un **grafo de vectores** donde cada nodo apunta a sus vecinos más cercanos:

**Vecinos más cercanos:**

$$
neighbors(v) = { u in V : || v - u || mínimo }
$$

Búsqueda rápida usando niveles jerárquicos:

$$
search(q) ≈ argmin_{v in G} || q - v ||
$$


In [7]:
import hnswlib

In [8]:
class IndexBuilder:
    def __init__(self, vectors):
        self.vectors = vectors
        self.index = None

    def build_ivf_index(self, n_clusters=10):
        """
        Construye un índice IVF simulado agrupando vectores en clusters.
        En la simulación usamos:

        cluster_id = i % n_clusters  ----> para asignar un vector a algún cluster de forma rápida. Sino utilizar otro algoritmo como K-means
        """
        clusters = {i: [] for i in range(n_clusters)}
        for i, vec in enumerate(self.vectors):
            cluster_id = i % n_clusters
            clusters[cluster_id].append(i)
        self.index = clusters
        print(f" Índice IVF simulado con {n_clusters} clusters creado.")


    def build_hnsw_index(self, ef_construction=200, M=16):
        """
        Construye un índice HNSW real utilizando la biblioteca hnswlib.
        
        Parámetros:
        - ef_construction: factor de exploración durante la construcción del índice.
        - M: número máximo de conexiones por nodo.
        """
        dim = self.vectors.shape[1]  # Dimensionalidad de los vectores
        self.index = hnswlib.Index(space='l2', dim=dim)  # librería para construir el grafo 
        self.index.init_index(max_elements=self.vectors.shape[0], ef_construction=ef_construction, M=M)
        self.index.add_items(self.vectors)

        print(f"✅ Índice HNSW real creado con {self.vectors.shape[0]} vectores.")


In [9]:
import numpy as np

In [10]:

def query_engine(vectors, query_vector, top_k=5, metric='cosine'):
    """
    Función para calcular similitudes entre un vector de consulta y un conjunto de vectores.
    
    Parámetros:
    - vectors: np.ndarray, matriz de vectores (n_docs x dim)
    - query_vector: np.ndarray, vector de consulta (dim,)
    - top_k: int, número de vecinos más cercanos a devolver
    - metric: str, 'cosine' o 'euclidean'
    
    Retorna:
    - top_indices: índices de los top_k vectores más similares
    - top_scores: similitudes (coseno) o distancias (euclidiana) correspondientes
    """
    query_vector = np.array(query_vector)
    
    if metric == 'cosine':
        # Normalizamos vectores para similitud coseno
        vectors_norm = vectors / np.linalg.norm(vectors, axis=1, keepdims=True)
        query_norm = query_vector / np.linalg.norm(query_vector)
        # Similitud coseno = producto punto de vectores normalizados
        sims = vectors_norm @ query_norm
        # Obtenemos top_k índices
        top_indices = np.argsort(sims)[::-1][:top_k]
        top_scores = sims[top_indices]
        
    elif metric == 'euclidean':
        # Distancia euclidiana
        dists = np.linalg.norm(vectors - query_vector, axis=1)
        top_indices = np.argsort(dists)[:top_k]
        top_scores = dists[top_indices]
        
    else:
        raise ValueError("Métrica no soportada. Use 'cosine' o 'euclidean'.")
    
    return top_indices, top_scores


In [11]:
def create_metadata_store(csv_path):
    """
    Crea un MetadataStore a partir de un archivo CSV con columnas:
    id, categoria, titulo, contenido

    Devuelve un objeto MetadataStore listo para usar.
    """
    class MetadataStore:
        """
        Almacena los metadatos de los documentos:
        - Texto original
        - Etiquetas o categorías
        """
        def __init__(self, df, text_column='contenido', label_column='categoria'):
            self.ids = df['id'].tolist()
            self.titles = df['titulo'].tolist()
            self.texts = df[text_column].tolist()
            self.labels = df[label_column].tolist()
            self.df = df

        def get_text(self, idx):
            return self.texts[idx]

        def get_label(self, idx):
            return self.labels[idx]

        def get_title(self, idx):
            return self.titles[idx]

        def get_metadata(self, idx):
            return {
                'id': self.ids[idx],
                'title': self.titles[idx],
                'text': self.texts[idx],
                'label': self.labels[idx]
            }

    return MetadataStore(df)

## SOLUCIÓN 3

In [13]:
metadata_store = create_metadata_store(csv_path)

def consulta_usuario(query_text, top_k=5, metric='cosine', embedding_dim=512):
    """
    Convierte la consulta del usuario en un embedding aleatorio (simulado)
    y devuelve los top_k documentos más similares según el metric especificado.
    
    Parámetros:
    - query_text: str, texto de la consulta
    - top_k: int, número de documentos a devolver
    - metric: str, 'cosine' o 'euclidean'
    - embedding_dim: dimensión de los embeddings
    """

    query_vector = np.random.rand(embedding_dim)
    query_vector = query_vector / np.linalg.norm(query_vector)  

    top_indices, top_scores = query_engine(np.array(embeddings), query_vector, top_k=top_k, metric=metric)

    results = []
    for idx, score in zip(top_indices, top_scores):
        meta = metadata_store.get_metadata(idx)
        meta['score'] = float(score)
        results.append(meta)
    return results


consulta = input("Escribe tu consulta: ")
resultados = consulta_usuario(consulta, top_k=5, metric='cosine')

print("\nDocumentos más similares:\n")
for r in resultados:
    print(f"ID: {r['id']} | Título: {r['title']} | Categoría: {r['label']} | Score: {r['score']:.4f}")
    print(f"Contenido: {r['text']}\n")


Documentos más similares:

ID: 1 | Título: Avances en IA | Categoría: Tecnología | Score: 0.7541
Contenido: La inteligencia artificial está transformando la industria del software.

ID: 4 | Título: Mercados globales | Categoría: Economía | Score: 0.7525
Contenido: Los mercados bursátiles muestran volatilidad ante cambios geopolíticos.

ID: 5 | Título: Descubrimiento espacial | Categoría: Ciencia | Score: 0.7494
Contenido: Un nuevo telescopio detecta exoplanetas similares a la Tierra.

ID: 7 | Título: Vacunas innovadoras | Categoría: Salud | Score: 0.7468
Contenido: Se desarrollan vacunas de ARN para distintas enfermedades infecciosas.

ID: 3 | Título: Aprendizaje automático | Categoría: Educación | Score: 0.7441
Contenido: El machine learning se utiliza en plataformas educativas personalizadas.



In [14]:
import numpy as np

query_text = "Inteligencia artificial en educación"  # ejemplo de consulta
query_embedding = np.random.rand(embedding_dim)      # generar embedding aleatorio
query_embedding = query_embedding / np.linalg.norm(query_embedding)  # normalizar para coseno

print("Embedding simulado de la consulta:", query_embedding)


Embedding simulado de la consulta: [0.0243128  0.03072299 0.01975071 0.05915896 0.05984423 0.07015604
 0.0256633  0.07157663 0.01113393 0.00640138 0.06824134 0.00295631
 0.05897692 0.01405346 0.01166803 0.04404628 0.03916665 0.06660361
 0.07364459 0.02398538 0.0121638  0.06241533 0.04429045 0.04268773
 0.06246591 0.01678022 0.02425854 0.02376129 0.02496451 0.04049475
 0.03132388 0.05614609 0.04580093 0.0489862  0.06532018 0.03514034
 0.02623944 0.03430231 0.00828738 0.07113374 0.01847041 0.06173907
 0.06792487 0.01771912 0.04854409 0.06618524 0.01767381 0.00506286
 0.02316678 0.0238674  0.00071891 0.06887295 0.01477317 0.03742872
 0.02056687 0.00063804 0.03748704 0.04641648 0.06492509 0.06742209
 0.01285731 0.07175478 0.03079755 0.06552276 0.05570067 0.03460577
 0.0191483  0.00723083 0.01831884 0.04945894 0.05634269 0.04225931
 0.04036246 0.04301334 0.05300632 0.03424647 0.0271789  0.04585473
 0.02931685 0.03251228 0.02432429 0.0087599  0.05289064 0.05942214
 0.05714336 0.01837269 0.04

## SOLUCIÓN 4

In [15]:
def consulta_por_categoria(query_text, categoria, top_k=5, metric='cosine', embedding_dim=512):
    """
    Busca documentos similares a la consulta dentro de una categoría específica.

    Parámetros:
    - query_text: str, texto de la consulta
    - categoria: str, categoría a filtrar (ej. "Tecnología")
    - top_k: int, número de documentos a devolver
    - metric: str, 'cosine' o 'euclidean'
    - embedding_dim: dimensión de los embeddings
    """
    # Simular embedding de la consulta
    query_vector = np.random.rand(embedding_dim)
    query_vector = query_vector / np.linalg.norm(query_vector)

    # Filtramos documentos por categoría
    indices_categoria = [i for i, label in enumerate(metadata_store.df['categoria']) if label == categoria]
    if not indices_categoria:
        print(f"No se encontraron documentos en la categoría '{categoria}'")
        return []

    # Extraemos embeddings de los documentos filtrados
    vectors_filtrados = np.array([embeddings[i] for i in indices_categoria])

    # Calcular similitudes
    top_indices_local, top_scores = query_engine(vectors_filtrados, query_vector, top_k=top_k, metric=metric)

    # Mapear de vuelta a los índices originales
    top_indices = [indices_categoria[i] for i in top_indices_local]

    # Obtener metadatos y resultados
    results = []
    for idx, score in zip(top_indices, top_scores):
        meta = metadata_store.get_metadata(idx)
        meta['score'] = float(score)
        results.append(meta)
    return results


In [16]:
consulta = "Avances en inteligencia artificial"
categoria = "Tecnología"

resultados = consulta_por_categoria(consulta, categoria, top_k=3, metric='cosine')

print(f"\nTop documentos en la categoría '{categoria}':\n")
for r in resultados:
    print(f"ID: {r['id']} | Título: {r['title']} | Score: {r['score']:.4f}")
    print(f"Contenido: {r['text']}\n")



Top documentos en la categoría 'Tecnología':

ID: 6 | Título: Computación cuántica | Score: 0.7508
Contenido: Se logran avances en la estabilidad de los qubits.

ID: 1 | Título: Avances en IA | Score: 0.7498
Contenido: La inteligencia artificial está transformando la industria del software.



## SOLUCIÓN 5

In [17]:
import time

def consulta_con_tiempo(query_text, categoria=None, top_k=5, metric='cosine', embedding_dim=512):
    """
    Realiza la consulta y devuelve los resultados junto con el tiempo de ejecución.
    
    Parámetros:
    - query_text: str, texto de la consulta
    - categoria: str o None, filtrar por categoría si se desea
    - top_k: int, número de documentos a devolver
    - metric: str, 'cosine' o 'euclidean'
    - embedding_dim: int, dimensión de los embeddings
    """
    start_time = time.time()  

    query_vector = np.random.rand(embedding_dim)
    query_vector = query_vector / np.linalg.norm(query_vector)

    # Filtramos por categoría si se especifica
    if categoria:
        indices_categoria = [i for i, label in enumerate(metadata_store.df['categoria']) if label == categoria]
        if not indices_categoria:
            print(f"No se encontraron documentos en la categoría '{categoria}'")
            return [], 0.0
        vectors_filtrados = np.array([embeddings[i] for i in indices_categoria])
        top_indices_local, top_scores = query_engine(vectors_filtrados, query_vector, top_k=top_k, metric=metric)
        top_indices = [indices_categoria[i] for i in top_indices_local]
    else:
        top_indices, top_scores = query_engine(embeddings, query_vector, top_k=top_k, metric=metric)

    # Obtenemos metadatos
    results = []
    for idx, score in zip(top_indices, top_scores):
        meta = metadata_store.get_metadata(idx)
        meta['score'] = float(score)
        results.append(meta)

    end_time = time.time()  # Fin cronómetro
    tiempo = end_time - start_time

    return results, tiempo


In [18]:
consulta = "Avances en inteligencia artificial"
categoria = "Tecnología"  # o None para buscar en todas las categorías

resultados, tiempo = consulta_con_tiempo(consulta, categoria=categoria, top_k=5, metric='cosine')

print(f"\nTiempo de consulta: {tiempo:.4f} segundos\n")
print("Documentos más similares:\n")
for r in resultados:
    print(f"ID: {r['id']} | Título: {r['title']} | Score: {r['score']:.4f}")
    print(f"Contenido: {r['text']}\n")



Tiempo de consulta: 0.0000 segundos

Documentos más similares:

ID: 6 | Título: Computación cuántica | Score: 0.7442
Contenido: Se logran avances en la estabilidad de los qubits.

ID: 1 | Título: Avances en IA | Score: 0.7436
Contenido: La inteligencia artificial está transformando la industria del software.



In [19]:
def evaluar_metricas(query_text, categoria=None, top_k=5, embedding_dim=512):
    """
    Compara resultados usando similitud coseno y distancia euclidiana.
    
    Retorna:
    - resultados_cos: lista de documentos top_k usando coseno
    - resultados_euc: lista de documentos top_k usando euclidiana
    """
    # Consulta con coseno
    resultados_cos, tiempo_cos = consulta_con_tiempo(query_text, categoria=categoria, top_k=top_k,
                                                     metric='cosine', embedding_dim=embedding_dim)
    
    # Consulta con euclidiana
    resultados_euc, tiempo_euc = consulta_con_tiempo(query_text, categoria=categoria, top_k=top_k,
                                                     metric='euclidean', embedding_dim=embedding_dim)
    
    return {
        'cosine': {'results': resultados_cos, 'tiempo': tiempo_cos},
        'euclidean': {'results': resultados_euc, 'tiempo': tiempo_euc}
    }


In [20]:
consulta = "Avances en inteligencia artificial"
categoria = "Tecnología"

comparacion = evaluar_metricas(consulta, categoria=categoria, top_k=5)

print("\n--- Resultados con Cosine ---")
for r in comparacion['cosine']['results']:
    print(f"ID: {r['id']} | Título: {r['title']} | Score: {r['score']:.4f}")

print(f"Tiempo: {comparacion['cosine']['tiempo']:.4f} s\n")

print("--- Resultados con Euclidean ---")
for r in comparacion['euclidean']['results']:
    print(f"ID: {r['id']} | Título: {r['title']} | Score: {r['score']:.4f}")

print(f"Tiempo: {comparacion['euclidean']['tiempo']:.4f} s\n")



--- Resultados con Cosine ---
ID: 1 | Título: Avances en IA | Score: 0.7570
ID: 6 | Título: Computación cuántica | Score: 0.7359
Tiempo: 0.0010 s

--- Resultados con Euclidean ---
ID: 1 | Título: Avances en IA | Score: 12.3565
ID: 6 | Título: Computación cuántica | Score: 12.6018
Tiempo: 0.0000 s



Con embeddings aleatorios, la distancia euclidiana no tiene correlación semántica, solo mide separación matemática. Deberíamos utilizar TF-IDF por ejemplo para que tenga más relevancia.

## SOLUCIÓN 6

In [21]:
import time
import hnswlib
import numpy as np

In [22]:
# -------------------------------
# 1. Función para búsqueda lineal (sin índice)
# -------------------------------
def busqueda_lineal(query_vector, top_k=5, metric='cosine'):
    start = time.time()
    top_indices, top_scores = query_engine(np.array(embeddings), query_vector, top_k=top_k, metric=metric)
    tiempo = time.time() - start
    results = [metadata_store.get_metadata(idx) for idx in top_indices]
    return results, tiempo


In [23]:

# -------------------------------
# 2. Función para búsqueda usando IVF simulado
# -------------------------------
class IVFSearch:
    def __init__(self, vectors, n_clusters=10):
        self.vectors = vectors
        self.n_clusters = n_clusters
        self.clusters = {i: [] for i in range(n_clusters)}
        for i, vec in enumerate(vectors):
            cluster_id = i % n_clusters
            self.clusters[cluster_id].append(i)
    
    def search(self, query_vector, top_k=5, metric='cosine'):
        start = time.time()
        # Determinamos cluster (simulado)
        cluster_id = np.random.randint(0, self.n_clusters)  # en la simulación elegimos aleatorio
        indices_cluster = self.clusters[cluster_id]
        vectors_cluster = np.array([self.vectors[i] for i in indices_cluster])
        top_indices_local, top_scores = query_engine(vectors_cluster, query_vector, top_k=top_k, metric=metric)
        top_indices = [indices_cluster[i] for i in top_indices_local]
        tiempo = time.time() - start
        results = [metadata_store.get_metadata(idx) for idx in top_indices]
        return results, tiempo

In [24]:
# -------------------------------
# 3. Función para búsqueda usando HNSW real
# -------------------------------
class HNSWSearch:
    def __init__(self, vectors, ef_construction=200, M=16):
        self.dim = vectors.shape[1]
        self.index = hnswlib.Index(space='l2', dim=self.dim)
        self.index.init_index(max_elements=vectors.shape[0], ef_construction=ef_construction, M=M)
        self.index.add_items(vectors)
        self.index.set_ef(50)  # para búsqueda
        self.vectors = vectors
    
    def search(self, query_vector, top_k=5):
        start = time.time()
        labels, distances = self.index.knn_query(query_vector, k=top_k)
        tiempo = time.time() - start
        results = [metadata_store.get_metadata(idx) for idx in labels[0]]
        return results, tiempo

In [25]:
# -------------------------------
# 4. Comparación de tiempos
# -------------------------------

embedding_dim = embeddings.shape[1]
query_vector = np.random.rand(embedding_dim)
query_vector = query_vector / np.linalg.norm(query_vector)

# Sin índice
res_lineal, t_lineal = busqueda_lineal(query_vector)
# Con IVF
ivf_search = IVFSearch(embeddings, n_clusters=10)
res_ivf, t_ivf = ivf_search.search(query_vector)
# Con HNSW
hnsw_search = HNSWSearch(embeddings)
res_hnsw, t_hnsw = hnsw_search.search(query_vector)

print(f"Tiempo búsqueda lineal: {t_lineal:.6f} s")
print(f"Tiempo búsqueda IVF: {t_ivf:.6f} s")
print(f"Tiempo búsqueda HNSW: {t_hnsw:.6f} s")


Tiempo búsqueda lineal: 0.000000 s
Tiempo búsqueda IVF: 0.002571 s
Tiempo búsqueda HNSW: 0.000000 s
